In [34]:
from pandas import read_csv, DataFrame, concat
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import snowballstemmer
from stop_words import get_stop_words
import collections

stop_word = get_stop_words('en')
df = read_csv("./spam_ham_dataset.csv")
new_df = df

df.drop(['Unnamed: 0', 'label'],axis=1,inplace=True)

print(df.head(5)["text"][0])

Subject: enron methanol ; meter # : 988291
this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary
flow data provided by daren } .
please override pop ' s daily volume { presently zero } to reflect daily
activity you can obtain from gas control .
this change is needed asap for economics purposes .


In [ ]:
print(df.head(10))

In [ ]:
stemmer: snowballstemmer.EnglishStemmer = snowballstemmer.stemmer('english')
def preprocess(text, stem=False):
    text = "\n".join(text.split("\n")[1:])
    tokens = []
    for token in text.split():
        if token not in stop_word:
            if stem:
                tokens.append(stemmer.stemWord(token))
            else:
                tokens.append(token)
    return " ".join(tokens)
   

for index, text in enumerate(df["text"]):
    data = preprocess(text)
    df["text"][index] = data

print(df.head(5)["text"])

In [ ]:
def make_Dictionary1(emails):
    all_words = []       
    for mail in emails:    
        words = mail.split()
        all_words += words
    
    dictionary = collections.Counter(all_words)

    for item in dictionary.copy():
        if not item.isalpha() or len(item) == 0:
            del dictionary[item]
            
    dictionary = dictionary.most_common(3000)

    return dictionary

word_dict = make_Dictionary1(df["text"])

In [ ]:
import numpy as np

def extract_features(email_text_list): 
    features_matrix = np.zeros((len(email_text_list),3000))
    docID = 0
    for text in email_text_list:
        words = text.split()
        for word in words:
            wordID = 0
            for i,d in enumerate(word_dict.copy()):
                if d[0] == word:
                    wordID = i
                    features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1
    return features_matrix

In [ ]:
feature_cols = ["text"]
predicted_cols = ["label_num"]

x = df[feature_cols]
y = df[predicted_cols]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.33, random_state=42)

train_df = [x_train, y_train]
train_df_rows: DataFrame = concat(train_df, axis=1)
sorted_train_df = train_df_rows.sort_values(by="label_num")

train_features = extract_features(sorted_train_df["text"])

In [35]:
model = GaussianNB()
model.fit(train_features, sorted_train_df["label_num"])

GaussianNB()

In [ ]:
test_df = [x_test, y_test]
test_df_rows = concat(test_df, axis=1)
sorted_test_df = test_df_rows.sort_values(by="label_num")
test_features = extract_features(sorted_test_df["text"])

In [37]:
test_result = model.predict(test_features)
train_result = model.predict(train_features)

In [38]:
from sklearn import metrics
train_confusion_matrix = metrics.confusion_matrix(sorted_train_df["label_num"], train_result)
test_confusion_matrix = metrics.confusion_matrix(sorted_test_df["label_num"], test_result)

print("Training confusion matrix:", train_confusion_matrix)
print("Testing confusion matrix:", test_confusion_matrix)

print("Accuracy:", metrics.accuracy_score(sorted_test_df["label_num"], test_result))

Training confusion matrix: [[2280  146]
 [   0 1038]]
Testing confusion matrix: [[1174   72]
 [  19  442]]
Accuracy: 0.5811364967779731
992
